<h1><b>ĐỒ ÁN CUỐI KỲ NMKHDL</b></h1>
<h2><b>POKEDEX</b></h2>

Họ tên: Nguyễn Đăng Quang

MSSV: 18120527



# Giới thiệu sơ lược
##### Có thể bạn chưa biết, Pocket Monsters viết tắt là Pokemon, một series game cũng như anime đình đám, series game Pokemon phát triển bởi GameFreak hiện vẫn đang làm mưa làm gió trên các hệ máy Nitendo và trên thị trường game toàn cầu. Nhân dịp gen9 (Scarlet & Violet) của game vừa ra mắt, chúng ta sẽ điểm lại các pokemon từ những thế hệ trước và thông số cũng như đặc tính của chúng thông qua Pokedex. Ở đây, chúng ta sẽ sử dụng "https://pokemondb.net/pokedex/all" là trang web chứa dữ liệu Pokedex, hay nói cách khác là danh sách của toàn bộ Pokemon ĐÃ xuất hiện xuyên suốt series game này. Let's go!

# Install và import
- bs4 (Beautiful Soup): Một thư viện Python dùng để phân tích các tài liệu HTML và XML. Nó tạo ra một cây phân tích cú pháp cho các trang được phân tích cú pháp có thể được sử dụng để trích xuất dữ liệu từ HTML, rất hữu ích cho web scrapping.
- requests: Cho phép gửi các yêu cầu HTTP bằng Python.

In [ ]:
!pip install bs4
!pip install requests

In [ ]:
import bs4
import requests
import pandas as pd

# Thu thập và xử lý dữ liệu
### 1. Lấy danh sách các id riêng biệt của từng Pokemon
- **Lưu ý:** 

      - Có thể các Pokemon có rất nhiều dạng (vùng miền, mega evolution, mega X/mega Y, nguyên thủy (primal form), tấn công/phòng thủ,...), hình dạng Pokemon thay đổi nhưng id thì vẫn giữ nguyên, vì vậy sẽ xuất hiện trường hợp một id lặp lại nhiều lần, chúng ta chỉ xét hình dạng cơ bản của chúng nên chỉ cần lấy 1 id cho mỗi Pokemon thôi. 

In [ ]:
url = 'https://pokemondb.net/pokedex/all'

source = requests.get(url).text
soup = bs4.BeautifulSoup(source,'html.parser')

# Thu thập id của toàn bộ pokemon
pokemon_id = soup.findAll('span', {'class': 'infocard-cell-data'})

id = []

for i in range(len(pokemon_id)):
    id.append(pokemon_id[i].text)
    id[i] = int(id[i])

id = list(set(id))      # Sắp xếp và xóa các id bị trùng

### 2. Danh sách các url của từng Pokemon
- Chúng ta có thể dùng id hoặc tên của Pokemon để làm subdirectory của url tương ứng với Pokemon đó. 

##### Ví dụ: Pokemon có tên là Bulbasaur, id là 001:

https://pokemondb.net/pokedex/bulbasaur

https://pokemondb.net/pokedex/001 hay https://pokemondb.net/pokedex/1

In [ ]:
# Lấy danh sách urls cho web scrapping
urls = []
n = len(id)

for i in range(n):
    urls.append('https://pokemondb.net/pokedex/' + str(id[i]))

### 3. Thu thập dữ liệu
#### 3.1. Thông tin
- Lưu ý: ID của Pokemon sẽ được thêm vào sau cùng sau đó đặt ID làm cột index, ở đây chúng ta chỉ xét các thuộc tính như: tên, hệ, loài,...
- Sẽ có thêm các cơ sở dữ liệu riêng để mô tả tác dụng của các đặc tính cũng như đặc tính ẩn và tương khắc hệ của chúng
- Các thông tin của một Pokemon sẽ bao gồm:

            - name: Tên

            - types_list: Hệ của Pokemon, mỗi con sẽ mang 1 hệ hoặc song hệ

            - species: Loài

            - height: Chiều cao

            - weight: Cân nặng

            - ability: Đặc tính

            - hid_ability (hidden ability): Đặc tính ẩn

            - catch_rate: Tỉ lệ bắt trúng bằng Poke Ball, bị ảnh hưởng bởi rất nhiều yếu tố:
                    * HP hiện tại
                    * Loại bóng (Poke Ball, Great Ball, Ultra Ball,...), Master Ball có tỷ lệ thu phục là 100% với tất cả Pokemon
                    * Trạng thái hiện tại (Choáng, phỏng, nhiễm độc, ngủ, tê liệt,...)
        
            - base_fs (base friendship): Chỉ số thân thiện cơ bản, một số Pokemon có chỉ số thân thiện là 0, bọn này có vẻ không ưa con người cho lắm

            - base_exp: Số lượng exp cần thiết để lên 1 level bất kỳ được tính theo hàm mũ. Mỗi loại có công thức khác nhau và khá phức tạp:
                    * Ví dụ 1 công thức đơn giản nhất cho mọi người:
                        Loại 1 000 000: exp = n^3 
                        n là level cần lên.
                        -> Để lên lv3 Pokemon loại này cần tổng cộng là 27exp trừ số exp hiện có sẽ ra số exp cần phải luyện thêm
            
            - growth_rate: Tốc độ tăng trưởng:
                    * Erratic: 600.000 exp ở level 100
                    * Fast: 800.000 điểm kinh nghiệm ở cấp 100
                    * Medium Fast: 1.000.000 điểm kinh nghiệm ở cấp 100
                    * Medium Slow: 1.059.860 điểm kinh nghiệm ở cấp 100
                    * Slow: 1.250.000 điểm kinh nghiệm ở cấp 100
                    * Fluctuating: 1.640.000 exp ở cấp 100:
        
            - genders: Giới tính theo tỷ lệ

            - Base Stats: Chỉ số cơ bản, bao gồm: 
                    * hp (Health Point): Sinh mệnh
                    * atk (Attack): Tấn công
                    * defn (Defense): Phòng thủ, giáp (Vì def là khai báo hàm nên không dùng để khai báo biến cho Defense được :v)
                    * sp_atk (Special Attack): Tấn công đặc biệt, sát thương phép
                    * sp_def (Special Defense): Phòng thủ đặc biệt, kháng phép
                    * spd (Speed): Tốc độ (tương tự như các game xoay tua, Pokemon tốc độ cao hơn sẽ được đánh trước)
                    
            - total: Tổng base stats


- Hàm get_pokemon_info(soup) giúp lấy ra thông tin của một Pokemon có ID chỉ định. Sau đó sẽ dùng vòng lặp để lấy thông tin toàn bộ Pokemon trong Pokedex

In [278]:
# Thu thập dữ liệu nào
def get_pokemon_info(soup):
    # name
    name = soup.find('h1').text

    # types
    types_list = soup.find_all('td')[1].text.replace('\n', '')      # 'type1 type2 '
    types_list = types_list[:-1].split(' ')                         # 'type1 type2'

    # type1, type2
    type1 = types_list[0]
    
    if (len(types_list) == 2):   # Pokemon mang song hệ
        type2 = types_list[1]
    else: 
        type2 = ''              # Pokemon mang 1 hệ

    # species
    species = soup.find_all('td')[2].text

    # height
    height = soup.find_all('td')[3].text.replace('\xa0', '')

    # weight
    weight = soup.find_all('td')[4].text.replace('\xa0', '')

    # ability
    ability = soup.find_all('span', {'class': 'text-muted'})[0].text.replace('1. ', '')

    # extra ability
    if '2. ' in soup.findAll(class_='text-muted')[1].text:
        ex_ability = soup.findAll(class_='text-muted')[1].text.replace('2. ', '')
    else: 
        ex_ability = ''
        
    
    # hidden ability
    hid_ability = soup.find_all('small', {'class': 'text-muted'})[0].text.replace(' (hidden ability)', '')
    if '(' in hid_ability:  # Pokemon không có hidden ability nên bị đọc nhầm sang khu vực xuất hiện
        hid_ability = ''

    # catch_rate
    catch_rate = soup.find_all('td')[8].text.replace('\n', '')

    # base friendship 
    base_fs = soup.find_all('td')[9].text.replace('\n', '')

    # base exp
    base_exp = soup.find_all('td')[10].text.replace('\n', '')

    # growth_rate
    growth_rate = soup.find_all('td')[11].text.replace('\n', '')

    # genders
    genders = soup.find_all('td')[13].text.replace('\n', '')

    # base stats (base, min, max): health point, attack, defense, special attack, special defense, speed, total (base)
    hp = int(soup.find_all('td')[15].text.replace('\n', ''))

    atk = int(soup.find_all('td')[19].text.replace('\n', ''))

    defn = int(soup.find_all('td')[23].text.replace('\n', ''))

    sp_atk = int(soup.find_all('td')[27].text.replace('\n', ''))
    
    sp_def = int(soup.find_all('td')[31].text.replace('\n', ''))

    spd =  int(soup.find_all('td')[35].text.replace('\n', ''))

    total = int(soup.find_all('td')[39].text)

    return name, type1, type2, species, height, weight, ability, ex_ability, hid_ability, catch_rate, base_fs, base_exp, growth_rate, genders, hp, atk, defn, sp_atk, sp_def, spd, total

#### 3.2. Tương khắc hệ
- Tương khắc hệ (Type Defenses) của một Pokemon là khả năng chịu đòn của một Pokemon trước những đòn tấn công mang hệ khác. Các Pokemon Trainer sẽ dựa vào tương khắc hệ này của chúng nhằm chọn ra Pokemon có ưu thế về hệ (Type Advantages) tốt nhất để đối đầu với Pokemon của đối thủ.

        Ví dụ: Bulbasaur mang song hệ là Grass/Poison:
                - Fire -> Grass/Poison = 2 <=> Đòn đánh hệ Fire sẽ gây gấp đôi sát thương lên Pokemon này
                - Grass -> Grass/Poison = 1/4   <=> Đòn đánh hệ Grass sẽ gây 1/4 sát thương lên Pokemon này

- Hàm type_defense(soup) sẽ trả về tương khắc hệ của Pokemon có url tương ứng. 

In [275]:
def type_defense(soup):
    type_defenses = []

    for i in range(40, 58):
        type_defenses.append(soup.findAll('td')[i].text)

    return type_defenses

#### 3.3. Đặc tính
- Đặc tính (Ability) và đặc tính ẩn (Hidden Ability): Là những đặc tính được trao cho mỗi Pokémon để có thể hỗ trợ chúng trong trận chiến, chúng ta sẽ lấy thông tin mô tả ngắn gọn toàn bộ đặc tính ở trang "https://pokemondb.net/ability".
- Một vài lợi ích đi kèm như: Thay đổi thời tiết, cường hóa tuyệt chiêu hoặc chỉ số, miễn nhiễm sát thương, giảm tấn công,...
 
        - Ví dụ: 
             * 'Intimidate' giảm chỉ số Attack của đối phương
             * 'Levitate' là trạng thái lơ lửng của Pokemon cho phép chúng miễn nhiễm với các đòn tấn công hệ Đất (Ground)
                
- Theo "https://pokemondb.net/ability", đặc tính và đặc tính ẩn đều gộp chung lại thành một danh sách được gọi là "The ability list".
- Danh sách này chứa:

                - Name: Tên đặc tính và đặc tính ẩn
                - Pokémon: Số lượng Pokemon sở hữu đặc tính đó
                - Description: Mô tả ngắn gọn về dặc tính đó
                - Gen: Thế hệ mà đặc tính đó được giới thiệu lần đầu

- Chúng ta chỉ cần quan tâm Name và Description.
- Hàm get_abilities_description(soup) trả về kết quả mô tả ngắn gọn về các Ability và Hidden Ability tương tứng.
- Lưu ý: 

        - Một vài Pokemon KHÔNG sở hữu Hidden Ability vì thế nội dung mô tả sẽ để trống
        - Hàm này chỉ trả về tên và mô tả các đặc tính ĐÃ ĐƯỢC GIỚI THIỆU XUYÊN SUỐT SERIES GAME POKEMON chứ không phải của từng Pokemon.

In [276]:
def get_abilities_description():
    source = requests.get('https://pokemondb.net/ability').text
    soup = bs4.BeautifulSoup(source,'html.parser')
    
    n1 = len(soup.findAll('tr')) - 1
    ability = []
    ability_des = []

    for i in range(n1):
        ability.append(soup.find_all('a', {'class': 'ent-name'})[i].text)
        ability_des.append(soup.find_all('td', {'class': 'cell-med-text'})[i].text)

    # blanked values
    ability.append('')
    ability_des.append('')
    return {k: v for k, v in zip(ability, ability_des)}

#### 3.4. Bắt đầu lấy thông tin, tương khắc hệ, đặc tính của từng Pokemon

In [279]:
data = []      # Thông tin
type_def = []  # Tương khắc hệ

for url in urls:
    source = requests.get(url).text
    soup = bs4.BeautifulSoup(source,'html.parser')
    # Lấy thông tin của từng Pokemon
    data.append(get_pokemon_info(soup))

    # Tương khắc hệ của từng Pokemon
    type_def.append(type_defense(soup))

# Mô tả đặc tính
abilities = get_abilities_description()

#### 3.4.1. Cơ sở dữ liệu thông tin Pokemon
- Sau khi đã có danh sách thông tin của toàn bộ Pokemon trong Pokedex, khởi tạo một cơ sở dữ liệu để lưu trữ
- Index ở đây sẽ là ID của từng Pokemon, bắt đầu từ 1.

In [280]:
pokedex = pd.DataFrame(data = data, columns=['Name', 'Type 1', 'Type 2', 'Species', 'Height', 'Weight', 'Ability', 'Extra Ability', 'Hidden Ability',
                                             'Catch Rate', 'Base Friendship', 'Base Exp', 'Growth Rate', 'Gender',
                                             'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Total'], index = id)
pokedex

,Name,Type 1,Type 2,Species,Height,Weight,Ability,Extra Ability,Hidden Ability,Catch Rate,...,Base Exp,Growth Rate,Gender,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Total
1,Bulbasaur,Grass,Poison,Seed Pokémon,0.7m (2′04″),6.9kg (15.2lbs),Overgrow,,Chlorophyll,"45 (5.9% with PokéBall, full HP)",...,64,Medium Slow,"87.5% male, 12.5% female",45,49,49,65,65,45,318
2,Ivysaur,Grass,Poison,Seed Pokémon,1.0m (3′03″),13.0kg (28.7lbs),Overgrow,,Chlorophyll,"45 (5.9% with PokéBall, full HP)",...,142,Medium Slow,"87.5% male, 12.5% female",60,62,63,80,80,60,405
3,Venusaur,Grass,Poison,Seed Pokémon,2.0m (6′07″),100.0kg (220.5lbs),Overgrow,,Chlorophyll,"45 (5.9% with PokéBall, full HP)",...,236,Medium Slow,"87.5% male, 12.5% female",80,82,83,100,100,80,525
4,Charmander,Fire,,Lizard Pokémon,0.6m (2′00″),8.5kg (18.7lbs),Blaze,,Solar Power,"45 (5.9% with PokéBall, full HP)",...,62,Medium Slow,"87.5% male, 12.5% female",39,52,43,60,50,65,309
5,Charmeleon,Fire,,Flame Pokémon,1.1m (3′07″),19.0kg (41.9lbs),Blaze,,Solar Power,"45 (5.9% with PokéBall, full HP)",...,142,Medium Slow,"87.5% male, 12.5% female",58,64,58,80,65,80,405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,Ursaluna,Ground,Normal,Peat Pokémon,2.4m (7′10″),290.0kg (639.3lbs),Guts,Bulletproof,Unnerve,"20 (2.6% with PokéBall, full HP)",...,—,Medium Fast,"50% male, 50% female",130,140,105,45,80,50,550
902,Basculegion,Water,Ghost,Big Fish Pokémon,3.0m (9′10″),110.0kg (242.5lbs),Swift Swim,Adaptability,Mold Breaker,"45 (5.9% with PokéBall, full HP)",...,—,Medium Fast,"100% male, 0% female",120,112,65,80,75,78,530
903,Sneasler,Poison,Fighting,Free Climb Pokémon,1.3m (4′03″),43.0kg (94.8lbs),Pressure,Unburden,Poison Touch,"20 (2.6% with PokéBall, full HP)",...,—,Medium Slow,"50% male, 50% female",80,130,60,40,80,120,510
904,Overqwil,Dark,Poison,Pin Cluster Pokémon,2.5m (8′02″),60.5kg (133.4lbs),Poison Point,Swift Swim,Intimidate,"45 (5.9% with PokéBall, full HP)",...,—,Medium Fast,"50% male, 50% female",85,115,95,65,65,85,510


#### 3.4.2. Cơ sở dữ liệu tương khắc hệ
- Tạo cơ sở dữ liệu để lưu trữ chỉ số tương khắc hệ.
- Chuẩn hóa dữ liệu dòng và thêm cột 'Name' chứa tên các Pokemon.

In [281]:
# Tạo csdl chứa tương khắc hệ
type_defenses = pd.DataFrame(data = type_def, columns = ['NORMAL', 'FIRE', 'WATER', 'ELECTRIC', 'GRASS', 'ICE', 
                                        'FIGHTING', 'POISON', 'GROUND', 'FLYING', 'SPYCHIC', 'BUG',
                                        'ROCK', 'GHOST', 'DRAGON', 'DARK', 'STEEL', 'FAIRY'], index = id)

# Thay đổi giá trị các dòng cho dễ nhìn
wrong = ['', '½', '¼']
right = ['1', '0.5', '0.25']

for i, j in zip(wrong, right):
    type_defenses = type_defenses.replace(i ,j)

# Thêm tên vào cơ dở dữ liệu
type_defenses.insert(0, 'Name', pokedex['Name'])

type_defenses

,Name,NORMAL,FIRE,WATER,ELECTRIC,GRASS,ICE,FIGHTING,POISON,GROUND,FLYING,SPYCHIC,BUG,ROCK,GHOST,DRAGON,DARK,STEEL,FAIRY
1,Bulbasaur,1,2,0.5,0.5,0.25,2,0.5,1,1,2,2,1,1,1,1,1,1,0.5
2,Ivysaur,1,2,0.5,0.5,0.25,2,0.5,1,1,2,2,1,1,1,1,1,1,0.5
3,Venusaur,1,2,0.5,0.5,0.25,2,0.5,1,1,2,2,1,1,1,1,1,1,0.5
4,Charmander,1,0.5,2,1,0.5,0.5,1,1,2,1,1,0.5,2,1,1,1,0.5,0.5
5,Charmeleon,1,0.5,2,1,0.5,0.5,1,1,2,1,1,0.5,2,1,1,1,0.5,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,Ursaluna,1,1,2,0,2,2,2,0.5,1,1,1,1,0.5,0,1,1,1,1
902,Basculegion,0,0.5,0.5,2,2,0.5,0,0.5,1,1,1,0.5,1,2,1,2,0.5,1
903,Sneasler,1,1,1,1,0.5,1,0.5,0.5,2,2,4,0.25,0.5,1,1,0.5,1,1
904,Overqwil,1,1,1,1,0.5,1,1,0.5,2,1,0,1,1,0.5,1,0.5,1,1


#### 3.4.3. Cơ sở dữ liệu mô tả các đặc tính
- Ý tưởng thực hiện:

                - B1: Trước tiên tạo database mới với tên, đặc tính, đặc tính ẩn Pokemon từ database cũ.
                - B2: Tạo 3 list là abi_des, ex_abi_des và hid_abi_des.
                - B3: Vì đặc tính 'Sharpness' vẫn chưa cập nhập mô tả trên "https://pokemondb.net/ability". Chúng ta sẽ cập nhập thủ công cho đặc tính này.
                - B4: Chạy vòng lặp từ 1 đến n+1 ứng với id từ 1 -> 905  (n = 905)
                - B5: Tạo các biến tạm để lưu giữ các Ability theo id
                - B6: abilities[temp] trả về mô tả ngắn gọn Ability tương ứng của Pokemon đó, tương tự như temp2 và temp3 với Extra Abiltiy và Hidden Ability.
                - B7: Tạo thêm 3 cột để chứa dữ liệu của 3 list.
                - B8: In csdl ra màn hình để kiểm tra.

In [283]:
# Tạo csdl chứa tên và đặc tính tương ứng của từng Pokemon
abilities_info = pokedex[['Name', 'Ability', 'Extra Ability', 'Hidden Ability']]     # B1

# B2
abi_des = []        # ability description
ex_abi_des = []     # extra ability description
hid_abi_des = []    # hidden ability description

# B3
abilities['Sharpness'] = 'Powers up slicing moves'

# B4
for i in range(1, n+1):
    # B5
    temp = abilities_info['Ability'][i]
    temp2 = abilities_info['Extra Ability'][i]
    temp3 = abilities_info['Hidden Ability'][i]
    
    # B6
    abi_des.append(abilities[temp])
    ex_abi_des.append(abilities[temp2])
    hid_abi_des.append(abilities[temp3])


# B7
abilities_info.insert(2, 'Ability Description', abi_des)
abilities_info.insert(4, 'Extra Ability Description', ex_abi_des)
abilities_info.insert(6, 'Hidden Ability Description', hid_abi_des)

# B8
abilities_info


,Name,Ability,Ability Description,Extra Ability,Hidden Ability Description,Hidden Ability
1,Bulbasaur,Overgrow,Powers up Grass-type moves in a pinch.,,Boosts the Pokémon's Speed in sunshine.,Chlorophyll
2,Ivysaur,Overgrow,Powers up Grass-type moves in a pinch.,,Boosts the Pokémon's Speed in sunshine.,Chlorophyll
3,Venusaur,Overgrow,Powers up Grass-type moves in a pinch.,,Boosts the Pokémon's Speed in sunshine.,Chlorophyll
4,Charmander,Blaze,Powers up Fire-type moves in a pinch.,,"In sunshine, Sp. Atk is boosted but HP decreases.",Solar Power
5,Charmeleon,Blaze,Powers up Fire-type moves in a pinch.,,"In sunshine, Sp. Atk is boosted but HP decreases.",Solar Power
...,...,...,...,...,...,...
901,Ursaluna,Guts,Boosts Attack if there is a status problem.,Bulletproof,Makes the foe nervous and unable to eat Berries.,Unnerve
902,Basculegion,Swift Swim,Boosts the Pokémon's Speed in rain.,Adaptability,Moves can be used regardless of Abilities.,Mold Breaker
903,Sneasler,Pressure,The Pokémon raises the foe's PP usage.,Unburden,May poison targets when a Pokémon makes contact.,Poison Touch
904,Overqwil,Poison Point,Contact with the Pokémon may poison the attacker.,Swift Swim,Lowers the foe's Attack stat.,Intimidate


# Xuất dữ liệu đã đọc được vào các file csv

In [284]:
pokedex.to_csv('pokemon_db.csv', encoding='utf-8', index = True)
type_defenses.to_csv('type_defenses.csv', encoding='utf-8', index = True)
abilities_info.to_csv('abilities_description.csv', encoding='utf-8', index = True)

# Q & A

### 1. Các Pokemon phân bố theo hệ như thế nào?

### 2.

### 3.

### 4.

### 5.